In [1]:
from utils import *
import re
from pprint import pprint
from nltk.corpus import stopwords

all_data = get_data_from_db()
all_data = all_data[(all_data["labels"]== "PO") | (all_data["labels"]== "NG")]

num_remover = NumRemover()
word_remover = WordRemover(stopwords.words('portuguese'))
all_data = num_remover.fit_transform(all_data)
all_data = word_remover.fit_transform(all_data)

neg_data = all_data[all_data["labels"] == "NG"]
pos_data = all_data[all_data["labels"] == "PO"]

ratio = 0.7
neg_train = neg_data.iloc[0:round(ratio*neg_data.shape[0]), :]
pos_train = pos_data.iloc[0:round(ratio*pos_data.shape[0]), :]

neg_test = neg_data.iloc[round(ratio*neg_data.shape[0]): , :]
pos_test = pos_data.iloc[round(ratio*neg_data.shape[0]): , :]

stopwords_pt = stopwords.words("portuguese")

# Vocabulários

In [10]:
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words= stopwords_pt)

cv.fit(neg_train["texts"])
vocab_neg = set(cv.vocabulary_.keys())

cv.fit(pos_train["texts"])
vocab_pos = set(cv.vocabulary_.keys())
# vocab_neg = set(' '.join(neg_train["texts"].tolist()).split())
# vocab_pos = set(' '.join(pos_train["texts"].tolist()).split())

print("Dimensão vocabulario negativo: " + str(len(vocab_neg)))
print("Dimensão vocabulario positivo: " + str(len(vocab_pos)))

Dimensão vocabulario negativo: 2630
Dimensão vocabulario positivo: 2272


In [11]:
# Verificar interseção dos vocabulários
intersect = vocab_neg.intersection(vocab_pos)
vocab_neg = vocab_neg.difference(intersect)
vocab_pos = vocab_pos.difference(intersect)


print("Interseção: ", len(intersect))

lexicon = list(vocab_neg.union(vocab_pos))

Interseção:  1084


In [12]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(2,2), stop_words= stopwords_pt, binary= True)),
                    ("lexicon_vector", CountVectorizer(vocabulary= lexicon, binary = False))
                    ])
folds = 10;
evaluate(all_data, features, folds)

# print("MultinomialNB")
# run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

# print("\n\nSVM")
# run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

# print("\nMaxent")
# run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

Naive Bayes---------------------------------
Cross Validation:
Accuracia media:  0.815955734406
Desvio padrão:  0.0208149938779

MaxEnt--------------------------------------
Cross Validation:
Accuracia media:  0.824527162978
Desvio padrão:  0.0256455898406

SVM-----------------------------------------
Cross Validation:
Accuracia media:  0.781750503018
Desvio padrão:  0.0460324225265


In [13]:
# Avaliando a incidência dos bigramas
cv = CountVectorizer(ngram_range=(1,2))

neg_counts = cv.fit_transform(neg_train["texts"])
neg_bigrams = cv.vocabulary_

pos_counts = cv.fit_transform(pos_train["texts"])
pos_bigrams = cv.vocabulary_

sp = np.sum(pos_counts.toarray(), axis=0)
sn = np.sum(neg_counts.toarray(), axis=0)

pos_bigrams  = {bigram: sp[index] for bigram, index in pos_bigrams.items()}
neg_bigrams  = {bigram: sn[index] for bigram, index in neg_bigrams.items()}

pos_excl_bigrams = {key:value for key, value in pos_bigrams.items() if key not in neg_bigrams}
neg_excl_bigrams = {key:value for key, value in neg_bigrams.items() if key not in pos_bigrams}

lexicon_bigrams = list(pos_excl_bigrams.keys()) + list(neg_excl_bigrams.keys())
len(lexicon_bigrams)

15430

In [14]:
from pandas import Series
s = Series(neg_excl_bigrams)
s.nlargest(15)

minas energia       15
edison              12
edison lobão        12
sabia               12
alves               11
sabesp              11
alves pmdb          10
câmara              10
eduardo alves       10
henrique eduardo    10
morto               10
não sabia           10
pelo ex             10
vaccari             10
calheiros            9
dtype: int64

In [16]:
from sklearn.naive_bayes  import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

features = FeatureUnion([
#                     ("bigram", CountVectorizer(ngram_range=(1,1), stop_words= stopwords_pt, vocabulary  = lexicon)),
                    ("lexicon_vector", CountVectorizer(binary= True, ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams))
                    ])

folds = 10;
print("MultinomialNB")
run_cross_validation(all_data, features, MultinomialNB(), n_folds= folds)

print("\n\nSVM")
run_cross_validation(all_data, features, SVC(C=316), n_folds=folds)

print("\nMaxent")
run_cross_validation(all_data, features, LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False), n_folds=folds)

MultinomialNB
Cross Validation:
Accuracia media:  0.851649899396
Desvio padrão:  0.0293590327385


SVM
Cross Validation:
Accuracia media:  0.69615694165
Desvio padrão:  0.0337314147193

Maxent
Cross Validation:
Accuracia media:  0.837303822938
Desvio padrão:  0.0391693402297


### Usando o PCA

In [28]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline, FeatureUnion

pca = PCA()
pca = TruncatedSVD(n_components=500)
cv  = CountVectorizer(ngram_range=(1,2), stop_words=stopwords_pt, vocabulary= lexicon_bigrams)

t_data = cv.fit_transform(all_data["texts"])
t_data = pca.fit_transform(t_data.toarray())

labels = np.array([])
for label in all_data["labels"]:
    if label == 'PO':
        labels = np.append(labels, [1])
    elif label == "NG":
        labels = np.append(labels, [-1])
    elif label == "NE":
        labels = np.append(labels, [0])

labels.shape

classifier = LogisticRegressionCV(fit_intercept=False, penalty= 'l2', dual= False)
# classifier = MultinomialNB()

print("Cross Validation:")
accuracy_average = np.array([])
sKFold = StratifiedKFold(n_splits= 10, shuffle= True, random_state= True)
for index, (train, test) in enumerate(sKFold.split(t_data, labels)):
    # Treinando um modelo Naive Bayes
    train_data = t_data[train]
    test_data = t_data[test]

    classifier.fit(train_data, labels[train])
    predictions = classifier.predict(test_data)

    accuracy = accuracy_score(labels[test], predictions, normalize=True)
    # classifier_models.append(pipeline)

    accuracy_average = np.append(accuracy_average, accuracy)
    #print("Fold ", index, " - Acuracia: ", accuracy)

print("Accuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Cross Validation:
Accuracia media:  0.848752515091
Desvio padrão:  0.0358056246919


Consegui um resultado equivalente com uma dimensionalidade menor